## Station data

In [33]:
import pandas as pd
from pathlib import Path

data_dir = Path('../data/2020-09')
station_data_path = data_dir / f"metadata_stations_{data_dir.name}.tsv"
station_df = pd.read_csv(station_data_path, sep='\t')

# Get min and max of location
ltt_min, ltt_max = station_df['Latitude [deg]'].min(), station_df['Latitude [deg]'].max()
lgt_min, lgt_max = station_df['Longitude [deg]'].min(), station_df['Longitude [deg]'].max()
print(f"Latitude  [deg]: [{ltt_min}, {ltt_max}]")
print(f"Longitude [deg]: [{lgt_min}, {lgt_max}]")

# list of lat long
latlong_list = list(zip(station_df['Latitude [deg]'], station_df['Longitude [deg]']))

Latitude  [deg]: [47.613, 49.612]
Longitude [deg]: [7.655, 10.773]


## Import data from google earth

In [2]:
import ee
ee.Authenticate()
ee.Initialize()

In [3]:
select_bands = ['dewpoint_temperature_2m', 'temperature_2m', 'skin_temperature',
        'soil_temperature_level_1', 'soil_temperature_level_2',
        'soil_temperature_level_3', 'soil_temperature_level_4',
        'skin_reservoir_content', 'volumetric_soil_water_layer_1',
        'volumetric_soil_water_layer_2', 'volumetric_soil_water_layer_3',
        'volumetric_soil_water_layer_4', 'surface_latent_heat_flux_sum',
        'surface_net_solar_radiation_sum', 'surface_net_thermal_radiation_sum',
        'surface_sensible_heat_flux_sum',
        'surface_solar_radiation_downwards_sum',
        'surface_thermal_radiation_downwards_sum', 'u_component_of_wind_10m',
        'v_component_of_wind_10m', 'surface_pressure',
        'leaf_area_index_high_vegetation', 'leaf_area_index_low_vegetation']

In [25]:
# Define the region of interest (roi)
roi = ee.Geometry.Rectangle([ltt_min, lgt_min, ltt_max, lgt_max])

# Define start and end date for a one-month period
start_date = ee.Date('2020-09-01')
end_date = ee.Date('2020-10-01')

# Generate a list of dates within the range (each day)
date_list = ee.List.sequence(0, end_date.difference(start_date, 'day').subtract(1)).map(
    lambda n: start_date.advance(n, 'day')
)

# Function to get reduced value for each day
def getDailyReducedValue(date):
    date = ee.Date(date)
    daily_data = ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR').select(
        select_bands).filterBounds(roi).filterDate(date, date.advance(1, 'day'))
    
    point = ee.Geometry.Point([ltt_min, lgt_min])
    
    reduced_value = daily_data.mean().reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point,
        scale=1
    )
    
    # Add the date to the result
    return ee.Dictionary(reduced_value).set('date', date.format('YYYY-MM-dd'))

# Map the function over the date list
daily_values = date_list.map(getDailyReducedValue)

# Get the info (Note: Depending on the size of the request, you might need to export this)
daily_values_info = daily_values.getInfo()

# Print the results
print(daily_values_info)

[{'date': '2020-09-01', 'dewpoint_temperature_2m': 288.62911224365234, 'leaf_area_index_high_vegetation': 0, 'leaf_area_index_low_vegetation': 1.2265625, 'skin_reservoir_content': 2.719461917876809e-07, 'skin_temperature': 304.42772102355957, 'soil_temperature_level_1': 304.7221972147624, 'soil_temperature_level_2': 304.2473882039388, 'soil_temperature_level_3': 303.0677687327067, 'soil_temperature_level_4': 302.89564514160156, 'surface_latent_heat_flux_sum': -236064, 'surface_net_solar_radiation_sum': 17472660, 'surface_net_thermal_radiation_sum': -7471594, 'surface_pressure': 94939.2236328125, 'surface_sensible_heat_flux_sum': -9610316, 'surface_solar_radiation_downwards_sum': 22665284, 'surface_thermal_radiation_downwards_sum': 34325071, 'temperature_2m': 301.79579798380536, 'u_component_of_wind_10m': 4.052747090657552, 'v_component_of_wind_10m': 5.363037308057149, 'volumetric_soil_water_layer_1': 0.08091163635253906, 'volumetric_soil_water_layer_2': 0.14442825317382812, 'volumetric

In [38]:
# Define the region of interest (roi) and the list of latitude/lgtgitude tuples
roi = ee.Geometry.Rectangle([ltt_min, lgt_min, ltt_max, lgt_max])
ltt_lgt_ee_list = ee.List(latlong_list)  # Convert to an Earth Engine list

# Define start and end date for a one-month period
start_date = ee.Date('2020-09-01')
end_date = ee.Date('2020-09-30')

# Generate a list of dates within the range (each day)
date_list = ee.List.sequence(0, end_date.difference(start_date, 'day').subtract(1)).map(
    lambda n: start_date.advance(n, 'day')
)

# Function to get reduced value for each day and each lat/lng tuple
def getDailyReducedValueByPoint(date):
    date = ee.Date(date)
    
    # Function to get reduced value for each lat/lng tuple
    def getReducedValueForPoint(ltt_lgt):
        lat = ee.Number(ee.List(ltt_lgt).get(0))
        lgt = ee.Number(ee.List(ltt_lgt).get(1))
        point = ee.Geometry.Point([lgt, lat])

        daily_data = ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR').select(
            select_bands).filterBounds(roi).filterDate(date, date.advance(1, 'day'))

        reduced_value = daily_data.mean().reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=point,
            scale=1
        )

        # Add date and lat/lng info to the result
        return ee.Dictionary(reduced_value).set('date', date.format('YYYY-MM-dd')).set('lat', lat).set('lgt', lgt)
    
    # Map over all lat/lng tuples (converted to EE List)
    return ltt_lgt_ee_list.map(getReducedValueForPoint)

# Map the function over the date list to get values for each date and lat/lng tuple
daily_values = date_list.map(getDailyReducedValueByPoint)

# Flatten the list of lists into a single list
flattened_values = daily_values.flatten()

# Get the info (Note: Depending on the size of the request, you might need to export this)
daily_values_info = flattened_values.getInfo()

# Print the results
print(daily_values_info)

[{'date': '2020-09-01', 'dewpoint_temperature_2m': 285.12732187906903, 'lat': 48.667, 'leaf_area_index_high_vegetation': 2.411773681640625, 'leaf_area_index_low_vegetation': 2.9012908935546875, 'lgt': 9.399, 'skin_reservoir_content': 0.0004215426743030548, 'skin_temperature': 286.80076789855957, 'soil_temperature_level_1': 288.1797981262207, 'soil_temperature_level_2': 289.0354741414388, 'soil_temperature_level_3': 290.7794386545817, 'soil_temperature_level_4': 289.38734436035156, 'surface_latent_heat_flux_sum': -4280744, 'surface_net_solar_radiation_sum': 7354328, 'surface_net_thermal_radiation_sum': -3429406, 'surface_pressure': 97467.7236328125, 'surface_sensible_heat_flux_sum': -739700, 'surface_solar_radiation_downwards_sum': 8697568, 'surface_thermal_radiation_downwards_sum': 29693031, 'temperature_2m': 286.9721488952637, 'u_component_of_wind_10m': -0.21295928955078125, 'v_component_of_wind_10m': -0.9430336554845175, 'volumetric_soil_water_layer_1': 0.3953469594319661, 'volumetri

In [39]:
gee_df = pd.DataFrame(daily_values_info)
# gee_df.to_csv(f'../gee_data/gee-{data_dir.name}.csv', index=False)
gee_df

,date,dewpoint_temperature_2m,lat,leaf_area_index_high_vegetation,leaf_area_index_low_vegetation,lgt,skin_reservoir_content,skin_temperature,soil_temperature_level_1,soil_temperature_level_2,...,surface_sensible_heat_flux_sum,surface_solar_radiation_downwards_sum,surface_thermal_radiation_downwards_sum,temperature_2m,u_component_of_wind_10m,v_component_of_wind_10m,volumetric_soil_water_layer_1,volumetric_soil_water_layer_2,volumetric_soil_water_layer_3,volumetric_soil_water_layer_4
0,2020-09-01,285.127322,48.667,2.411774,2.901291,9.399,0.000422,286.800768,288.179798,289.035474,...,-739700,8697568.0,29693031,286.972149,-0.212959,-0.943034,0.395347,0.362453,0.231876,0.341794
1,2020-09-01,284.907107,48.830,2.293874,2.936707,9.196,0.000327,287.063463,288.380644,289.235751,...,-1063552,9271782.0,29687437,287.082663,-0.225248,-0.957804,0.370071,0.329721,0.228356,0.339532
2,2020-09-01,284.965864,49.245,2.192749,2.966705,8.641,0.000187,289.336331,290.163034,290.674878,...,-2425476,13241368.0,29438341,288.963197,-0.703438,-1.347249,0.308034,0.209976,0.217313,0.334862
3,2020-09-01,284.639285,49.356,2.181249,2.801768,8.532,0.000191,290.007799,290.818551,291.343091,...,-1947476,13431636.0,29275101,289.625957,-0.512845,-1.737386,0.290041,0.195446,0.213290,0.338119
4,2020-09-01,285.219200,48.009,2.771647,3.095103,7.835,0.000401,287.558825,288.778594,289.372144,...,-937692,13182968.0,29223051,287.833558,-0.147937,-1.395955,0.379819,0.345477,0.235288,0.339797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1155,2020-09-29,283.397323,48.787,2.159307,2.714742,8.189,0.000296,284.643357,285.519854,286.453453,...,469741,1838475.5,31471104,284.954437,1.591626,2.031487,0.360085,0.300435,0.220308,0.346844
1156,2020-09-29,281.848251,48.649,2.195002,2.838923,8.654,0.000102,283.690557,284.380856,285.099449,...,-444275,2554003.5,30815796,283.761891,1.963086,1.389071,0.318976,0.297829,0.245399,0.333718
1157,2020-09-29,282.067489,47.624,2.299276,2.936452,8.255,0.000309,283.489060,284.500648,285.341474,...,255673,3042989.5,30360932,283.836191,1.737418,0.860181,0.395058,0.384167,0.334979,0.352047
1158,2020-09-29,281.845158,48.803,2.143448,2.950190,9.800,0.000228,282.483689,283.420732,284.479088,...,298493,1988593.5,30046732,283.229827,1.482292,0.841098,0.444759,0.427835,0.386208,0.435826
